### This assignment is following the instructions that provided by Dr. Bianco. 

It is finished by Yunhe Cui as an individual task (though I use "we" instead of "me").


# !!! Please run the spatial_join_in_py3.ipynb in a python3 environment before rerun this notebook!!!



### test if the distribution of 

#### 1) trip duration of bikers that ride during the day vs night

#### 2) age of bikers for trips originating in Manhattan and in Brooklyn (extra credit)

are different. Use 3 tests: KS, Pearson's, Spearman's. 

Use the scipy.stats functions scipy.stats.ks_2samp, scipy.stats.pearsonr, scipy.stats.spearmanr. 

For the KS do the test with the entire dataset and with a subset 200 times smaller

Choose a single significant threshold for the whole exercise. 

For each test phrase the Null Hypothesis in words.

Describe the return of the scipy function you use in each case.

State the result in terms of rejection of the Null.

In [ ]:
import pylab as pl
import pandas as pd
import numpy as np
#imports downloader
from getCitiBikeCSV import getCitiBikeCSV
import scipy.stats
import os
%pylab inline

# the getCitiBikeCSV is retrieved from Dr. Bianco's folder in repo PUI2018/HW06

### Read in Data

In [ ]:
# using the citibike data for 2015-02 (cold month) and 2018-08 (warm month)
datastring_1 = '201502'
getCitiBikeCSV(datastring_1)
datastring_2 = '201508'
getCitiBikeCSV(datastring_2)

In [ ]:
!ls $PUIDATA
# files are now in the PUIDATA folder as listed

In [ ]:
df_02 = pd.read_csv("%s/201502-citibike-tripdata.csv"%os.getenv("PUIDATA"))
df_08 = pd.read_csv("%s/201508-citibike-tripdata.csv"%os.getenv("PUIDATA"))


In [ ]:
df = pd.concat([df_02, df_08])

In [ ]:
# print ("df_02 shape is {}".format(df_02.shape))
# print ("df_08 shape is {}".format(df_08.shape))
# print ("the combined df shape is {}".format(df.shape))

## Problem 1: trip duration of bikers that ride during the day vs night

#### Since we used the ridership data from Feb and Aug, the sunrise-sunset times are different at a large scale. In this case, we define the day time as 7am to 7pm and night time from 7pm to 7am of the following day.
H0: There is NO statistical difference in term of trip duration depending on time (day/night)   
Ha: There is statistical difference in term of trip duration depending on time (day/night)   
significance level α = 0.05

In [ ]:
df['date'] = pd.to_datetime(df['starttime'])
df['hour'] = df['date'].dt.hour


In [ ]:
df.columns

In [ ]:
df.head()

### split by category

In [ ]:
df.columns

In [ ]:
# drop redundant columns
df_c = df.drop([u'starttime', u'stoptime', u'start station name',u'end station name',u'bikeid',u'usertype',u'birth year', u'gender', u'date'], axis = 1)
df_c = df_c.reset_index()


In [ ]:
# give the day/night attribute
df_c['dur_day'] = df_c['tripduration'][(df_c['hour']>4) & (df_c['hour']<=17)]
df_c['dur_night'] = df_c['tripduration'][(df_c['hour']<=4) | (df_c['hour']>17)]

In [ ]:
df_c = df_c.iloc[:,1:]
#df_c

In [ ]:
# get the description
df_c.describe()

In [ ]:
#dropping the NaN value
df_c['dur_day'].dropna(inplace= True)
df_c['dur_night'].dropna(inplace= True)
df_c.head()

### plot the data in histograms

In this case, we use 60 sec as a split bin. The maximum data is too large which could be considered as outliers. We could use 1 hour (3600 second) as the upper boundary of our dataset to show the trend of our data   
The histograms are as follow

In [ ]:
bins = np.arange(0, 3600, 60)

Day = df_c.dur_day.groupby(pd.cut(df_c.dur_day, bins)).agg([count_nonzero]).plot(kind='bar', figsize = (20,4),legend=False)
Day.set_title("Day Trip Durations")

Night = df_c.dur_night.groupby(pd.cut(df_c.dur_night, bins)).agg([count_nonzero]).plot(kind='bar',figsize = (20,4),legend=False)
Night.set_title("Night Trip Durations");

Figure1. This is a histogram of Day/Night citibike ride duration distribution. The raw data is from citibike 2015 Feb and Aug. We could see that the distribution of those day/night trip duration have similar positive skewed trend. However, the number of day ridership is more than 3 times of night ones.  

In [ ]:
#print df.ageS, df.ageS.cumsum()

drd=df_c.dur_day.groupby(pd.cut(df_c.dur_day, bins)).agg([count_nonzero]).cumsum()

drn=df_c.dur_night.groupby(pd.cut(df_c.dur_night, bins)).agg([count_nonzero]).cumsum()

print (np.abs(drd / drd.max()- drn / drn.max()))

pl.plot(bins[:-1] + 5, drd / drd.max(), label = "Day")
pl.plot(bins[:-1] + 5, drn / drn.max(), label = "Night")
pl.plot(bins[:-1] + 5, np.sqrt(drn / drn.max() - drd / drd.max())**2, 'k-',label = "difference")
pl.xlabel("Trip duration")
pl.ylabel("Normalized Cumulative Number")
pl.legend()

Figure2. This chart shows the normalized day/night citibike trip duration. It is clear that the difference between the distribution of them is very small. 

### 1.1-1 test to compare 2 samples
According to https://docs.scipy.org/doc/scipy0.15.1/reference/generated/scipy.stats.ks_2samp.html, If the K-S statistic is small or the p-value is high, then we cannot reject the hypothesis that the distributions of the two samples are the same. Only the 2-sided test could be implemented. 

In [ ]:
ks = scipy.stats.ks_2samp(df_c.dur_day, df_c.dur_night)
print (ks)

#### We got p-value is extremely small (we could see it as 0) which is definitely smaller than 0.05 (our preset significance level). In this case, we COULD REJECT the Null hypothesis which is "There is NO statistical difference in term of trip duration depending on time (day/night)"

### 1.1-2 KS test w/ reduced dataset

In [ ]:
df_r = df_c.sample(df_c['dur_day'].shape[0]/200, axis = 0)
df_r['dur_day'].dropna(inplace= True)
df_r['dur_night'].dropna(inplace= True)
ks_r = scipy.stats.ks_2samp(df_r.dur_day, df_r.dur_night)
print (ks_r)

#### In this case, we used a random sample which is selected from the dataset (sample size  200 times smaller). The p value is 0.2701 which is larger than 0.05. Thus, we could not reject the null hypothesis. 

The scipy.stats KS test already tells me the significance and the p-value.  
The next few cells are here just to show you how you would obtain the same result by hand, but they are not required.  
Remember: the Null hypothesis is rejected if  
$D_KS(n1,n2) &gt; c(\alpha) \sqrt{\frac{(n1 + n2)}{n1n2}}$
(see class notes) where $c(\alpha$) is the inverse of the KS distribution, and you do not have to know how to get that cause there are tables that list critical values!!  
http://www.real-statistics.com/tests-normality-and-symmetry/statistical-tests-normality-symmetry/kolmogorov-smirnov-test/kolmogorov-distribution/  
But also this result depends in your choice of binning through, and thustheresultyou get by hand may not be exactly the same as the one the KS returns. Either way: this is how you would calculate the KS statistics by hand.

### 1.2 Pearson's test for correlation
#### -> the following words are retrieved from scipy.org   
Like other correlation coefficients, this one varies between -1 and +1 with 0 implying no correlation. Correlations of -1 or +1 imply an exact linear relationship. Positive correlations imply that as x increases, so does y. Negative correlations imply that as x increases, y decreases.  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr

### H0: There is NO linear relationship between Day&Night trip duration
significance level α = 0.05

In [ ]:
# scipy.stats.pearsonr(x , y) x,y should be the same length
len_min = min(df_c['dur_day'].shape, df_c['dur_night'].shape)
len_min = len_min[0]

In [ ]:
Day_reduced = np.sort(df_c['dur_day'].sample(n = len_min, axis = 0))
Night_reduced = np.sort(df_c['dur_night'].sample(n = len_min, axis = 0))

In [ ]:
pearson = scipy.stats.pearsonr(Day_reduced,Night_reduced)
pearson

### The correlation coefficient of Pearson's test is 0.95966, which indicate a strong positive linear relationship. The p value is 0.0 and we could reject our null hypothesis. 

### 1.3 Spearman's test for correlation
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy.stats.spearmanr  


In [ ]:
# scipy.stats.spearmanr(a, b=None, axis=0, nan_policy='propagate')
spearman = scipy.stats.spearmanr(Day_reduced,Night_reduced)
spearman

### The correlation coefficient of Spearman's test almost 1, which indicate a strong positive linear relationship. The p value is 0.0 and we could reject our null hypothesis. 

=======================================================================
## Problem 2: age of bikers for trips originating in Manhattan and in Brooklyn (extra credit)

# !!! Please run the spatial_join_in_py3.ipynb in a python3 environment before rerun this notebook!!!


####  H0: There is no statistical difference in the age distribution of bikers in Manhattan and Brooklyn
#### significance level p = 0.05

In [ ]:
df['age'] = 2015 - df['birth year']
df.head()

In [ ]:
!ls $PUIDATA

In [ ]:
borough = pd.read_csv("%s/borough.csv"%os.getenv("PUIDATA"))

In [ ]:
borough.head()

In [ ]:
borough = borough.rename(columns = {'StationNam': 'start station name'})

In [ ]:
df_b = pd.merge(df, borough, on='start station name')


In [ ]:
df_b.columns

In [ ]:
df_b = df_b.drop([u'starttime', u'stoptime', u'start station id',
       u'start station name', u'start station latitude',
       u'start station longitude', u'end station id', u'end station name',
       u'end station latitude', u'end station longitude', u'bikeid',
       u'usertype', u'birth year', u'gender', u'date', u'hour',
       u'Latitude', u'Longitude', u'boro_code'], axis = 1)

In [ ]:
df_b.sort(['boro_name'])

In [ ]:
Df_Manh = df_b.loc[(df_b['boro_name'] == 'Manhattan')]
Df_Manh = Df_Manh.reset_index().iloc[:,1:]
Df_Manh.dropna(inplace = True)
Df_Manh.head()

In [ ]:
Df_Bkly = df_b.loc[(df_b['boro_name'] == 'Brooklyn')]
Df_Bkly = Df_Bkly.reset_index().iloc[:,1:]
Df_Bkly.dropna(inplace = True)
Df_Bkly.head()

### The above steps are linking borough name and citibike data based on the START station name. Then we cleaned the data for conducting the correlation.  


### Plot the data in histograms

In [ ]:
bins_2 = np.arange(10, 99, 5)
MN = Df_Manh.age.groupby(pd.cut(Df_Manh.age, bins_2)).agg([count_nonzero]).plot(kind='bar',legend=False)
MN.set_title("Manhattan riders")
BK = Df_Bkly.age.groupby(pd.cut(Df_Bkly.age, bins_2)).agg([count_nonzero]).plot(kind='bar',legend=False)
BK.set_title("Brooklyn riders");

Figure3. This is a histogram of Manhattan/Brooklyn citibike rider age distribution. The raw data is from citibike 2015 Feb and Aug. We could see that the distribution of those Manhattan/Brooklyn rider age duration have similar positive skewed trend while the Brooklyn one is more concentrate near the axis. 

In [ ]:
agM = Df_Manh.age.groupby(pd.cut(Df_Manh.age, bins_2)).agg([count_nonzero]).cumsum()

agB = Df_Bkly.age.groupby(pd.cut(Df_Bkly.age, bins_2)).agg([count_nonzero]).cumsum()

print (np.abs(agM / agM.max() - agB / agB.max()))

pl.plot(bins_2[:-1] + 5, agM / agM.max(), label = "Manhattan")
pl.plot(bins_2[:-1] + 5, agB / agB.max(), label = "Brooklyn")
pl.plot(bins_2[:-1] + 5, np.sqrt(agB / agB.max() - agM / agM.max())**2, 'k-',label = "difference")
pl.xlabel("Age")
pl.ylabel("Normalized Cumulative Number")
pl.legend()

Figure2. This chart shows the normalized Manhattan/Brooklyn citibike rider age cumulative distribution. It is clear that the difference between the distribution of them is generally very small. However, in the age range 30-50, there is a relative large difference between those two categories. 

#### Skeleton ipynb: 

"They look similar! But the difference gets to 10%. If I wanted to code the KS test by hand I woud have everything I need: the normalized cumulative distributions can be subtracted from each other and the max distance can calculated.
Notice that there may be NaN values you are gonna have to deal with! You can do that for example with a Boolean statementsuch as df.ageF[~np.isnan(df.ageF)] or you can use numpy functions that deal with Nan values: nansum, nanmean, nanstd...
lets run the scipy KS test"

###  2.1-1 KS test to compare 2 samples:

In [ ]:
ks_2 = scipy.stats.ks_2samp(Df_Manh.age, Df_Bkly.age)
print (ks_2)

### We got p-value = 0 which is smaller than 0.05. In this case, we COULD REJECT the Null hypothesis which is "There is NO statistical difference in the age distribution of bikers in Manhattan and Brooklyn"

### 2.1-2 KS test w/ reduced dataset

In [ ]:
Df_Manh.shape[0]

In [ ]:
Df_Bkly.shape[0]

In [ ]:
# Df_Manh_r = Df_Manh.sample(1017560/200, axis = 0)
# Df_Bkly_r = Df_Bkly.sample(67166/200, axis = 0)
#Df_Manh.dropna(inplace = True)

In [ ]:
ks_r_2 = scipy.stats.ks_2samp(Df_Manh.sample(1017560/200, axis = 0), Df_Bkly_r)
print (ks_r_2)

#### In this case, we used a random sample which is selected from the dataset . The p value is 0.2701 which is larger than 0.05. Thus, we could not reject the null hypothesis. 

### 1.2 Pearson's test for correlation